<a href="https://colab.research.google.com/github/sudhirshahu51/RAG/blob/main/pdf_query_with_Hybrid_Search_and_cross_reranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index langchain chromadb pypdf llama-index-llms-groq llama-index-embeddings-huggingface llama-index-vector-stores-chroma --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.3 MB/s eta 0:00:0

In [ ]:
!pip install llama-index-retrievers-bm25 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.7/646.7 kB 12.3 MB/s eta 0:00:00


In [ ]:
# to make async work well with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# **1. Text Extraction from PDF file**

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/Data/microsoft_annual_report_2022.pdf"]).load_data()

In [ ]:
documents[5].text

'4 \n  \nOur commitment to responsibly develop and use technologies like AI is core to who we are. We put our commitment into \npractice, not only within Microsoft but by empowering our customers and partners to do the same and by advocating for \npolicy change. We released our Responsible AI Standard, which outlines 17 goals aligned to our six AI principles and \nincludes tools and practices to support them. And we share our open -source tools, including the new Responsible AI \nDashboard, to help developers building AI technologies identify and mitigate issues before deployment.  \nFinally, we provide clear reporting and information on how we run our business and how we work with customers and \npartners, delivering the transparency that is central to trust. Our annual Impact Summary shares more about our progress \nand learnings across these four commitments, and our Reports Hub  provides detailed reports on our environmental data, \nour political activities, our workforce demograph

In [ ]:
len(documents)

93

# **2. Splitting Chunks**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from llama_index.core.node_parser import LangchainNodeParser          #understand sentence and token based splitting can also be done iusing llamaindex or wrapper of langchain in llamaindex.

In [ ]:
parser  = LangchainNodeParser(RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=512,
    chunk_overlap=100))
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
len(nodes)

684

# **3. Create embeddings**

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [ ]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import chromadb
# chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_20221")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model, show_progress=True, use_async=True)


Generating embeddings: 100%|██████████| 69/69 [05:37<00:00,  4.90s/it]  


In [ ]:
#index = VectorStoreIndex.from_documents(
#    documents, storage_context=storage_context, embed_model=embed_model, transformation = [SentenceSplitter(chunk_size=25, chunk_overlap=0),
#        TitleExtractor(),], show_progress=True, use_async=True
#)

# **4. Calling LLM using GROQ API**

In [ ]:
from google.colab import userdata
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
groq_key = userdata.get('GROQ_API')
llm = Groq(model="llama3-70b-8192", api_key=groq_key) #https://docs.llamaindex.ai/en/stable/examples/llm/groq/#streaming

TimeoutException: Requesting secret GROQ_API timed out. Secrets can only be fetched when running from the Colab UI.

In [ ]:
response = llm.complete("Explain the importance of low latency LLMs")

In [ ]:
print(response)

In [ ]:
response = llm.stream_complete("Explain the importance of low latency LLMs")

In [ ]:

for r in response:
    print(r.delta, end="")

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Interactive Systems**: In interactive systems like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, ensuring a seamless user experience. Delays in response can lead to user frustration and abandonment.
2. **Real-time Applications**: Low-latency LLMs are vital in real-time applications like language translation, sentiment analysis, and text summarization, where timely processing is critical. For instance, in live subtitles or real-time language translation, delays can cause confusion or miscommunication.
3. **Edge AI**: With the proliferation of edge AI, low-latency LLMs are necessary to process data closer to the source, reducing latency and improving performance. This is particularly important in applications like autonomous vehicles, smar

# **5. Query Engine**

### **i. BM25 Retriver**

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [ ]:
# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

DEBUG:bm25s:Building index from IDs objects


In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve("Measure revenue sources of mircrosoft?")

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 3132c5a7-cee2-415c-90f3-5d8d89411d01<br>**Similarity:** 2.5193252563476562<br>**Text:** impracticability of determining the geographic source of the revenue.  
Revenue, classified by significant product and service offerings, was as follows:  
  
(In millions)         
        
Year Ended June 30, 2022 2021 2020 
        
Server products and cloud services $ 67,321  $ 52,589  $ 41,379  
Office products and cloud services  44,862   39,872   35,316  
Windows  24,761   22,488   21,510  
Gaming  16,230   15,370   11,575  
LinkedIn  13,816   10,289   8,077<br>

**Node ID:** c67218f2-6ede-42a6-bef5-9b2d0aae3667<br>**Similarity:** 2.3889145851135254<br>**Text:** may incur future impairments.  
Equity investments with readily determinable fair values are measured at fair value. Equity investments without readily 
determinable fair values are measured using the equity method or measured at cost with adjustments for observable 
changes in price or impairments (referred to as the measurement alternative). We perform a qualitative assessment on a<br>

**Node ID:** 6fbc6d6e-5766-4657-b0da-9663954dbe8f<br>**Similarity:** 2.329604387283325<br>**Text:** GAAP measures aid investors by providing additional insight into our operational performance and help clarify trends 
affecting our business. For comparability of reporting, management considers non -GAAP measures in conjunction with 
GAAP financial results in evaluating business performance. These non-GAAP financial measures presented should not be 
considered a substitute for, or superior to, the measures of financial performance prepared in accordance with GAAP.<br>

**Node ID:** c934d185-d824-4799-9c11-e8e0b393f712<br>**Similarity:** 2.32543683052063<br>**Text:** source projects, including Eclipse (sponsored by CA Technologies, IBM, Oracle, and SAP), PHP, and Ruby on Rails.<br>

**Node ID:** 5bc745a2-7536-4b82-ab7a-87906f308800<br>**Similarity:** 2.2575418949127197<br>**Text:** recognized in other income (expense), net.  
Fair Value Measurements  
We account for certain assets and liabilities at fair value. The hierarchy below lists three levels of fair value based on th e 
extent to which inputs used in measuring fair value are observable in the market.  We categorize each of our fair value 
measurements in one of these three levels based on the lowest level input that is significant to the fair value measurement 
in its entirety. These levels are:<br>

### **ii. Vector Store Retriever**

In [ ]:
vector_retriever  = index.as_retriever(similarity_top_k=2, )

In [ ]:
# Query Data
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>The revenue sources of Microsoft can be measured by the following categories: Server products and cloud services, Office products and cloud services, Windows, Gaming, and LinkedIn.</b>

#**6. Hybrid Search**

In [ ]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

## **i. Hybrid Search using Reciprocal Rerank Fusion score.**

In [ ]:
from llama_index.core.retrievers import QueryFusionRetriever


retriever = QueryFusionRetriever(
        [vector_retriever,
        bm25_retriever],
        similarity_top_k=5, #Get top 5 nodes from naximum of 2*(5)(vector retriver) + 2*(5)(bm25) = 20 nodes. (5) for 5 artificial queries.
        num_queries = 5,  # 5 Artificial Query generation.
        use_async=True)

In [ ]:
# check how decide Alpha for Above

In [ ]:
nodes = retriever.retrieve("Measure revenue sources of mircrosoft?")

for node in nodes:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")
    display_source_node(node, source_length=5000)
    print("--------------------------------------------------")


Score: 6.42 - to access our content and services, and the ability to create new experiences through first-party content creators.  
Search and News Advertising  
Our Search and news advertising business is designed to deliver relevant search, native, and display advertising to a 
global audience. We have several partnerships with other companies, including Yahoo, through which we provide and 
monetize search queries. Growth depends on our ability to attract new users, understand intent, and match intent with...
-----



**Node ID:** a60419fd-aa57-4c4e-8977-7ba4114880af<br>**Similarity:** 6.423602104187012<br>**Text:** to access our content and services, and the ability to create new experiences through first-party content creators.  
Search and News Advertising  
Our Search and news advertising business is designed to deliver relevant search, native, and display advertising to a 
global audience. We have several partnerships with other companies, including Yahoo, through which we provide and 
monetize search queries. Growth depends on our ability to attract new users, understand intent, and match intent with<br>

--------------------------------------------------
Score: 4.58 - web.queries@computershare.com  
You can also send mail to the transfer agent at:  
Computershare  
P.O. Box 43006  
Providence RI 02940-3078  
Shareholders can sign up for electronic alerts to access the 
annual report and proxy statement online. The service gets 
you the information you need faster and also gives you the 
power and convenience of online proxy voting. To sign up for 
this free service, visit the Annual Report site on the Investor 
Relations website at:...
-----



**Node ID:** 9b17796b-59ce-4327-91ec-7d047fad0033<br>**Similarity:** 4.581233501434326<br>**Text:** web.queries@computershare.com  
You can also send mail to the transfer agent at:  
Computershare  
P.O. Box 43006  
Providence RI 02940-3078  
Shareholders can sign up for electronic alerts to access the 
annual report and proxy statement online. The service gets 
you the information you need faster and also gives you the 
power and convenience of online proxy voting. To sign up for 
this free service, visit the Annual Report site on the Investor 
Relations website at:<br>

--------------------------------------------------
Score: 3.99 - 84 
REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM  
To the Stockholders and the Board of Directors of Microsoft Corporation  
Opinion on the Financial Statements  
We have audited the accompanying consolidated balance sheets of Microsoft Corporation and subsidiaries (the 
“Company”) as of June 30, 2022 and 2021, the related consolidated statements of income, comprehensive income, cash...
-----



...

--------------------------------------------------
Score: 3.71 - across the Company. 
In addition to our main research and development operations, we also operate Microsoft Research. Microsoft Research is 
one of the world’s largest corporate research organizations and works in close collaboration with top universities around 
the world to advance the state-of-the-art in computer science and a broad range of other disciplines, providing us a unique 
perspective on future trends and contributing to our innovation....
-----



**Node ID:** e46b0772-2e49-4d37-8664-7475b6d74333<br>**Similarity:** 3.7086267471313477<br>**Text:** across the Company. 
In addition to our main research and development operations, we also operate Microsoft Research. Microsoft Research is 
one of the world’s largest corporate research organizations and works in close collaboration with top universities around 
the world to advance the state-of-the-art in computer science and a broad range of other disciplines, providing us a unique 
perspective on future trends and contributing to our innovation.<br>

--------------------------------------------------
Score: 3.15 - 89 
  
DIRECTORS AND EXECUTIVE OFFICERS OF MICROSOFT CORPORATION  
  
DIRECTORS  
  
Satya Nadella 
Chairman and Chief Executive Officer, 
Microsoft Corporation  
Sandra E. Peterson 2,3 
Operating Partner, 
Clayton, Dubilier & Rice, LLC 
John W. Stanton 1,4 
Founder and Chairman, Trilogy 
Partnerships 
      
Reid G. Hoffman 4 
General Partner, Greylock Partners 
Penny S. Pritzker 4 
Founder and Chairman, 
PSP Partners, LLC 
John W. Thompson 3,4 
Lead Independent Director, 
Microsoft Corporation...
-----



**Node ID:** 98381a47-de3a-40e0-b95b-24dcee35e926<br>**Similarity:** 3.153303861618042<br>**Text:** 89 
  
DIRECTORS AND EXECUTIVE OFFICERS OF MICROSOFT CORPORATION  
  
DIRECTORS  
  
Satya Nadella 
Chairman and Chief Executive Officer, 
Microsoft Corporation  
Sandra E. Peterson 2,3 
Operating Partner, 
Clayton, Dubilier & Rice, LLC 
John W. Stanton 1,4 
Founder and Chairman, Trilogy 
Partnerships 
      
Reid G. Hoffman 4 
General Partner, Greylock Partners 
Penny S. Pritzker 4 
Founder and Chairman, 
PSP Partners, LLC 
John W. Thompson 3,4 
Lead Independent Director, 
Microsoft Corporation<br>

--------------------------------------------------


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever)

In [ ]:
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>Based on the provided context, Microsoft's revenue sources can be measured from the following areas:

1. Search and News Advertising: Through partnerships with companies like Yahoo, Microsoft provides and monetizes search queries, generating revenue from relevant search, native, and display advertising.

2. Services: Microsoft offers services that assist customers in developing, deploying, and managing Microsoft server solutions, Microsoft desktop solutions, and Nuance conversational AI and ambient intelligent solutions, along with providing training and certification to developers and IT professionals on various Microsoft products.

3. OEMs (Original Equipment Manufacturers): Microsoft generates revenue from two broad categories of OEMs: regional and local OEMs, and system builders. These OEMs source Microsoft software for pre-installation and local redistribution.

4. Direct Agreements: Many organizations that license Microsoft products and services transact directly with the company through Enterprise Agreements and other direct relationships.</b>

## **ii\. Hybrid Search with query expansion and using cosine similarity for final nodes**

# Cross- ReRanker

# **Query Expansion**

In [ ]:
!pip install umap-learn --quiet

In [ ]:
import umap.umap_ as umap
import numpy as np
from tqdm import tqdm


*   Expansion with multiple queries



In [ ]:
def augment_multiple_query(query, llm):

    messages = [
    ChatMessage(
        role="system", content="You are a helpful expert financial research assistant. Your users are asking questions about an annual report. "
            "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
            "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
            "Make sure they are complete questions, and that they are related to the original question."
            "Output one question per line. Do not number the questions."
    ),
    ChatMessage(role="user", content=query ),
    ]
    resp = llm.chat(messages)
    return resp

In [ ]:
original_query = "What were the most important factors that contributed to increases in revenue?"
aug_queries = augment_multiple_query(original_query, llm)
print(aug_queries)


assistant: What was the total revenue for the year?
Which product or service segment generated the most revenue?
Were there any significant changes in pricing or sales volume?
Did the company experience any notable acquisitions or divestitures?
Were there any changes in the company's business model or strategy?


In [ ]:
augmented_queries = aug_queries.message.blocks[0].text.split("\n")
queries = [original_query] + augmented_queries

In [ ]:
vector_retriever  = index.as_retriever(similarity_top_k=2,)
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [ ]:
all_nodes = []

In [ ]:
for query in queries:
  nodes_v = vector_retriever.retrieve(original_query)
  nodes_b = bm25_retriever.retrieve(original_query)
  all_nodes = all_nodes + nodes_v + nodes_b

In [ ]:
unique_nodes = {}
for nod in all_nodes:
  if nod.node_id not in unique_nodes.keys():unique_nodes[nod.node_id] = nod.text

In [ ]:
# Query Data
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

1. Take 20 chunks using vector retriver for augumneted queries.
2. Take 20 chunks using BM25 retriver for augumneted queries.
3. remove the duplicates.
4.  Based u[on dot matrix product with query, choose top 5 and pass as context.

Now integrate re-ranker, graph search too and later on image search.  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.9 MB/s eta 0:00:00


# **6. Re-ranking**

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [ ]:
scores = cross_encoder.predict(pairs) #learn how cross encoders works

In [ ]:
print("Scores:")
for score in scores: print(score)

Scores:
-7.73989
-11.079268
-11.040518
-11.116722
-7.445792
-10.886526
-10.743448
-10.859211
-11.121365
-11.007851
-10.985082
-11.233842
-11.022106
-11.142147
-11.1664505
-7.503585
-11.05914
-4.6518903
-10.017991
-6.902089
-9.316702
-7.917177
-10.822462
-10.973504
-9.4284725
-11.11678
-4.3417664
-10.947717
-10.088552
-10.311901
-11.072828
-11.024439
-11.248009
-1.1369962
-11.141821
-9.768026
-11.04459
-10.187727
-4.8184843
-10.814183
-11.074697
-11.137989
-11.103973
-11.157962
-11.078432
-10.882423
-8.505108
-3.768155
-5.2747493
-10.119463


In [ ]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:print(o)

New Ordering:
33
47
26
17
38
48
19
4
15
0
21
46
20
24
35
18
28
49
37
29
6
39
22
7
45
5
27
23
10
9
12
31
2
36
16
30
40
44
1
42
3
25
8
41
34
13
43
14
11
32


Call LLM and use prompt template to pass conetxt and get the response and Also try reranker from llamindex directly and call llm using that.

In [ ]:
def rag(query, retrieved_documents, llm):
    information = "\n\n".join(retrieved_documents)

    messages = [
    ChatMessage(
        role="system", content="You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report. You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
    ),
    ChatMessage(role="user", content=f"Question: {query}. \n Information: {information}"),
    ]

    resp = llm.stream_chat(messages)
    return resp


In [ ]:
output = rag(query=query, retrieved_documents=retrieved_documents, llm = llm)

for r in output:
    print(r.delta, end="")

The total revenue was $198,270 million.

Now Use Evaluation and then Integrate Graph Search.

# **7. Apply Embedding Adaptors to improve the results more.**
from 5th tutorial.

# **8. Applying Hybrid Search**

[link](https://superlinked.com/vectorhub/articles/optimizing-rag-with-hybrid-search-reranking)